# Test ComCam photodiode

Craig Lage - 17-Jul-24

In [ ]:
import sys, asyncio, time, os
import pickle as pkl
import numpy as np
import matplotlib.pyplot as plt
from astropy.time import Time
from astropy.io import fits

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel.comcam import ComCam

## Now connect to ComCam and the Electrometer CSC

In [ ]:
# change this to a local directory
data_dir = '/home/cslage/DATA'

In [ ]:
logger = logging.getLogger(f"Daytime Checkout {Time.now()} UT")
logger.level = logging.DEBUG

In [ ]:
logger.info(os.environ["OSPL_URI"])
logger.info(os.environ["LSST_DDS_PARTITION_PREFIX"])

In [ ]:
domain = salobj.Domain()
comcam = ComCam(domain)
electrometer = salobj.Remote(name="Electrometer", domain=domain, index=101) # Check SAL index

In [ ]:
await electrometer.start_task
await comcam.start_task

In [ ]:
await comcam.get_available_filters()

In [ ]:
state = await comcam.get_state()

In [ ]:
state = salobj.State.STANDBY
tmp = await salobj.set_summary_state(electrometer, state,timeout=60)

In [ ]:
elec_tmp = await electrometer.evt_summaryState.aget()
print('Electrometer: ',salobj.State(elec_tmp.summaryState))

In [ ]:
elec_tmp.salIndex

In [ ]:
test = electrometer.evt_detailedState.get()
print(test)

In [ ]:
det_state = await electrometer.evt_detailedState.aget()

In [ ]:
det_state

In [ ]:
await comcam.take_flats(1.0, 1, reason="Test", program="Test")

In [ ]:
comcam.take_flats?

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)
await electrometer.cmd_setRange.set_start(setRange=2E-8) # -1 is autorange
await electrometer.cmd_setIntegrationTime.set_start(intTime=0.01) 

In [ ]:
mode = await electrometer.evt_measureType.aget()

In [ ]:
mode = await electrometer.evt_measureType.aget()
intTime = await electrometer.evt_integrationTime.aget()
range_ = await electrometer.evt_measureRange.aget()
print(f'Mode: {mode.mode}; Int. Time: {intTime.intTime}; Range: {range_.rangeValue}')

In [ ]:
await electrometer.cmd_setLogLevel.set_start(level=10)

In [ ]:
await electrometer.cmd_setRange.set_start(setRange=-1) # -1 is autorange

In [ ]:
await electrometer.cmd_setIntegrationTime.set_start(intTime=0.2) 

In [ ]:
await electrometer.cmd_startScanDt.set_start(scanDuration=10)

In [ ]:
lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
filename= os.path.split(lfa.url)[1]
print(filename)
elec_filen = os.path.join(data_dir, filename)
os.system(f"curl {lfa.url} --output {elec_filen}")

In [ ]:
# Look at Data
hdu = fits.open(f'{elec_filen}')
data = hdu[1].data
print(data)

## Bring in needed subroutines

In [ ]:
# functions to run the electrometer measurements
def get_file_transfer(lfa):
    filename = os.path.split(lfa.url)[1]
    save_dir = os.path.join(data_dir, filename.split(':')[0])
    if not os.path.exists(save_dir):
        os.mkdir(save_dir)
    save_file = os.path.join(save_dir, filename)
    os.system(f"curl {lfa.url} --output {save_file}")
    return save_file

async def elec_meas(exp_time):
    electrometer.evt_largeFileObjectAvailable.flush()
    tmp1=await electrometer.cmd_startScan.set_start(timeout=10)
    await asyncio.sleep(exp_time)
    tmp2=await electrometer.cmd_stopScan.set_start(timeout=10)
    lfa = await electrometer.evt_largeFileObjectAvailable.next(flush=False, timeout=10)
    filename = get_file_transfer(lfa)
    filename = filename.replace(":", "_")
    elec_filen = os.path.join(data_dir, filename)
    os.system(f"curl {lfa.url} --output {elec_filen}")
    hdu = fits.open(f'{elec_filen}')
    data = hdu[1].data
    return filename, lfa, data



## Set up electrometer

In [ ]:
await electrometer.cmd_performZeroCalib.set_start(timeout=10)
await electrometer.cmd_setDigitalFilter.set_start(activateFilter=False, activateAvgFilter=False, activateMedFilter=False, timeout=10)  
await electrometer.cmd_setIntegrationTime.set_start(intTime=0.1) 
await electrometer.cmd_setRange.set_start(setRange=-1)

## Take 20 biases for system warm-up

In [ ]:
# Take 20 biases for system warm-up
for i in range(20):
    await comcam.take_bias(1)
    await asyncio.sleep(2.0)


## Now run SDSSg filter


### 2024MMDDxxxxx is the first exposure

In [ ]:
latiss_grating= 'empty_1'
wave_step = 5 # Wavelength step in nm
latiss_exp_time = 60.0
num_latiss_exps = 1

# Dictionary to hold the results
results = {}
for latiss_filter in ['SDSSg_65mm']:#['SDSSg_65mm', 'SDSSr_65mm', 'SDSSi_65mm']:
    if latiss_filter =='SDSSg_65mm':
        min_wave = 350; max_wave = 600; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSr_65mm':
        min_wave = 500; max_wave = 750; grating = 1; slit_1 = 5.0; slit_2 = 5.0
    elif latiss_filter == 'SDSSi_65mm':
        min_wave = 640; max_wave = 880; grating = 2; slit_1 = 5.0; slit_2 = 5.0
    else:
        raise "Filter not supported"
    await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=min_wave, gratingType=grating, \
                                                             fontEntranceSlitWidth=slit_1, fontExitSlitWidth=slit_2, timeout=180)        
    wave = min_wave
    # This loop steps through the wavelengths for each filter
    while wave < max_wave + wave_step / 2.0:
        await atmonochromator.cmd_updateMonochromatorSetup.set_start(wavelength=wave, gratingType=grating, \
                                                             fontEntranceSlitWidth=slit_1, fontExitSlitWidth=slit_2, timeout=180)        
        key = f"{latiss_filter}_{wave:.0f}"
        print(key)
        filename, electrometer_lfa, electrometer_data = await elec_meas(1.0)
        for (t,current) in electrometer_data:
            electrometer_current = current
        exp_time = 0.5
        filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
        spec_max = np.max(spectrum)
        # The loop below prevents the spectrum from saturating
        while spec_max > 15900:
            exp_time /= 2.0
            filename, fiber_lfa, wavelengths, spectrum = await fiber_meas(exp_time)
            spec_max = np.max(spectrum)
        # Take one flat with the filter and one without
        await latiss.take_flats(latiss_exp_time, num_latiss_exps, filter=latiss_filter, grating=latiss_grating, reason=f'flat_{wave:.1f}')
        await latiss.take_flats(latiss_exp_time, num_latiss_exps, filter='empty_1', grating=latiss_grating, reason=f'flat_{wave:.1f}')
        
        print(wave, electrometer_current, exp_time, spec_max)
        results[key] = {}
        results[key]['electrometer_lfa_url']=electrometer_lfa.url
        results[key]['fiber_lfa_url']=fiber_lfa.url
        results[key]['electrometer_current']=electrometer_current
        results[key]['fiber_data']=[wavelengths, spectrum, exp_time]
        print(f"Done with {latiss_filter}, {wave}")
        wave += wave_step

In [ ]:
# Pickle the data
filename = "/scratch/cslage/filter_cals/filter_cal_30apr24.pkl"
file = open(filename, "wb")
pkl.dump(results, file)
file.close()

In [ ]:
# Turn off the lamp
tmp = await WhiteLightSource.cmd_turnLampOff.set_start()

## Below shows how to unpickle the spectrum and electrometer data and plot a spectrum

In [ ]:
# Unpickle the data
filename = "/scratch/cslage/filter_cals/filter_cal_30apr24.pkl"
file = open(filename, "rb")
results = pkl.load(file)
file.close()

In [ ]:
[wavelengths, spectrum, exp_time] = results['SDSSg_65mm_450']['fiber_data']

In [ ]:
%matplotlib inline
plt.plot(wavelengths, spectrum)
plt.xlim(350,450)

In [ ]:
current = results['SDSSg_65mm_450']['electrometer_current']
current